In [50]:
from bs4 import BeautifulSoup as bs
import requests
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import httpx

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
#options.add_argument('--incognito')
#options.add_argument('--headless')
options.add_argument('--disable-javascript')

driver = webdriver.Chrome(executable_path=r"C:\Users\Chloe\Documents\chromedriver_win32\chromedriver.exe",options=options)

C:\Users\Chloe\AppData\Local\Temp\ipykernel_24744\3007230380.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\Chloe\Documents\chromedriver_win32\chromedriver.exe",options=options)


In [51]:
df = pd.DataFrame({'title': pd.Series(dtype='str'),
                   'release_date': pd.Series(dtype='str'),
                   'rating': pd.Series(dtype='str'),
                   'critic_score': pd.Series(dtype='str'),
                   'audience_score': pd.Series(dtype='str'),
             
})

print(df)

#specifies the website to scrape and use selenium
driver.get("https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc")

WebDriverWait(driver, 1).until(
    lambda s: s.find_element(By.CLASS_NAME,"clamp-list").is_displayed()
)

i = 0
while True:
    try:
        # waits until the head-block class is displayed
        WebDriverWait(driver, 1).until(
            lambda s: s.find_element(By.CLASS_NAME,'clamp-list').is_displayed()
        )
    except TimeoutException:
        break

    url = driver.current_url
    session = requests.Session()
    session.max_redirects = 200
    response = session.get(url,headers={'User-Agent': 'Google Chrome'})
    #response = requests.get(url)
    #print(response.history)
    html = response.content
    #creates of beautiful soup object of the current url
    soup = bs(html, "lxml")
    #print(soup)
    
    #time.sleep(1)
    #makes a list of each div with a class of "item-info"
    all_item_info = soup.find_all("td", class_="clamp-summary-wrap")
    #for each item in the list, gather all the relevant data about the books
    for item_info in all_item_info:
        try:
            movie_title = item_info.find("a", class_="title").find("h3").get_text(strip=True)
        except:
            movie_title = pd.NA
        try:
            movie_release = item_info.find("div", class_="clamp-details").find_all("span")[-2].get_text(strip=True)
        except:
            movie_release = pd.NA
        try:
            movie_rating = item_info.find("div", class_="clamp-details").find_all("span")[-1].get_text(strip=True)
        except:
            movie_rating = pd.NA
        try:
            movie_crit_score = item_info.find("a", class_="metascore_anchor").find("div").get_text(strip=True)
        except:
            movie_crit_score = pd.NA
        try:
            movie_user_score = item_info.find("div", class_="browse-score-clamp").find("div", class_="clamp-userscore").find("a").find("div").get_text(strip=True)
        except:
            movie_user_score = pd.NA
            
        df = pd.concat([df,pd.Series({'title': movie_title,
                       'release_date': movie_release,
                       'rating': movie_rating,
                       'critic_score': movie_crit_score,
                       'audience_score': movie_user_score
                      }).to_frame().T], ignore_index=True)
    
    
    try:
        load_more = driver.find_element(By.XPATH,'//a[@rel="next"]')
        if load_more: driver.execute_script("arguments[0].click();", load_more)
    except: pass
        #break
    #after each page it increments i by 1, once i is greater than 33 we have reached
    #the last page so it breaks out of the while statement
    i += 1
    if i > 154:
        break
        
        
df.to_csv('metacritic_data.csv')
df  

Empty DataFrame
Columns: [title, release_date, rating, critic_score, audience_score]
Index: []


,title,release_date,rating,critic_score,audience_score
0,Tokyo Story,"March 13, 1972",| Not Rated,100,8.7
1,The Godfather,"March 24, 1972",| R,100,9.2
2,Citizen Kane,"September 4, 1941",| Approved,100,8.4
3,Rear Window,"September 1, 1954",| TV-G,100,8.7
4,Casablanca,"January 23, 1943",| TV-PG,100,8.8
...,...,...,...,...,...
15438,The Garbage Pail Kids Movie,"August 22, 1987",| PG,1,0.7
15439,United Passions,"June 5, 2015",| Not Rated,1,0.7
15440,Bio-Dome,"January 12, 1996",| PG-13,1,7.5
15441,Chaos,"August 12, 2005",| Not Rated,1,2.5
